In [ ]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/')
sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl')
import dl.loaders.ultrasound_dataset as ultrasound_dataset
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu

import importlib

from pytorch3d.ops import (knn_points, 
                           knn_gather)

sys.path.append('/mnt/raid/C1_ML_Analysis/source/ShapeAXI/src/')
from shapeaxi.saxi_layers import SelfAttention


In [ ]:
def plot_diffusion(X, P=None, frame=None, scale=10):
    num_surf = len(X)
    specs_r = [{'type': 'scatter3d'} for _ in range(num_surf)]

    fig = make_subplots(
        rows=1, cols=num_surf,
        specs=[specs_r]
    )

    for idx, x in zip(range(num_surf), X):
        # First scatter plot
        fig.add_trace(
            go.Scatter3d(x=x[:,0], y=x[:,1], z=x[:,2], mode='markers', marker=dict(
                size=2,
                color=x[:,2],                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.8
            )),
            row=1, col=idx+1
        )

    if P is not None:
        for idx, p in zip(range(num_surf), P):
            fig.add_trace(
                go.Scatter3d(x=p[:,0], y=p[:,1], z=p[:,2], mode='markers', marker=dict(
                    size=4,
                    color='red',                # set color to an array/list of desired values
                    opacity=1,
                )),
                row=1, col=idx+1
            )

    if frame is not None:
        for idx, f in zip(range(num_surf), frame):

            origin = P[idx,0]     

            for i in range(3):
                fig.add_trace(go.Scatter3d(
                    x=[origin[0], origin[0] + f[i, 0]*scale],
                    y=[origin[1], origin[1] + f[i, 1]*scale],
                    z=[origin[2], origin[2] + f[i, 2]*scale],
                    mode='lines',
                    line=dict(color='cyan', width=8),
                ), 
                row=1, col=idx+1)

    return fig

def compute_orthogonal_frame(points: torch.Tensor) -> torch.Tensor:    
    """
    Given a tensor of shape [B, 3, 3] representing three 3D points per batch,
    returns a tensor of shape [B, 3, 3] representing an orthogonal frame [x, y, z] for each batch.
    """
    p0 = points[:, 0]
    p1 = points[:, 1]
    p2 = points[:, 2]
    
    v1 = p1 - p0
    v2 = p2 - p0

    # Normalize x (first direction)
    x = F.normalize(v1, dim=1)

    # Compute z = normalized cross(v1, v2)
    z = F.normalize(torch.cross(v1, v2, dim=1), dim=1)

    # Compute y = cross(z, x)
    y = torch.cross(z, x, dim=1)

    # Stack the vectors as rows of the rotation matrix
    frame = torch.stack([x, y, z], dim=1)  # [B, 3, 3]

    return frame

In [ ]:
# fig = plot_diffusion(X_PC.cpu().numpy())
# fig.show()

In [ ]:
# p = torch.tensor([[0.04572, 0.6788, 0.5607], [-3.0951, 0.1676, -1.2842], [0.3316, -8.0302, 0.38299]]) # hands on head model. We picked a point on the head, the belly and shoulder 
# dist = knn_points(p.unsqueeze(0), X_PC, K=1)
# dist.idx
# idx = dist.idx

# idx = torch.tensor([[[1470],
#          [3369],
#          [2043]]])

# k = knn_gather(X_PC, idx).squeeze(2)

# frame = compute_orthogonal_frame(k)
# plot_diffusion(X_PC.cpu().numpy(), k.cpu().numpy(), frame.cpu().numpy())


In [ ]:
rest = np.load("/mnt/raid/C1_ML_Analysis/simulated_data_export/fetus_rest_selected/rest.npy")

head = np.load("/mnt/raid/C1_ML_Analysis/simulated_data_export/fetus_rest_selected/head.npy")
belly = np.load("/mnt/raid/C1_ML_Analysis/simulated_data_export/fetus_rest_selected/belly.npy")
side = np.load("/mnt/raid/C1_ML_Analysis/simulated_data_export/fetus_rest_selected/side.npy")

In [ ]:
rest_pc = torch.tensor(rest).to(torch.float32)
head_pc = torch.tensor(head).unsqueeze(0).to(torch.float32)
belly_pc = torch.tensor(belly).unsqueeze(0).to(torch.float32)
side_pc = torch.tensor(side).unsqueeze(0).to(torch.float32)

In [ ]:
side_pc.shape

In [ ]:

dist = knn_points(head_pc, rest_pc, K=1)
head_idx = dist.idx

head_k = knn_gather(rest_pc, head_idx).squeeze(2)

dist = knn_points(belly_pc, rest_pc, K=1)
belly_idx = dist.idx
belly_k = knn_gather(rest_pc, belly_idx).squeeze(2)

dist = knn_points(side_pc, rest_pc, K=1)
side_idx = dist.idx
side_k = knn_gather(rest_pc, side_idx).squeeze(2)

rest_points = torch.stack([torch.mean(belly_k, dim=1), 
                           torch.mean(head_k, dim=1),
                           torch.mean(side_k, dim=1)], dim=1)

frame_rest = compute_orthogonal_frame(rest_points)
plot_diffusion(rest_pc.cpu().numpy(), rest_points.cpu().numpy(), frame_rest.cpu().numpy(), scale=1)

In [ ]:
batch_size = 4
dm = ultrasound_dataset.ImgPCDataModule(
    mount_point="/mnt/raid/C1_ML_Analysis/simulated_data_export",
    csv_train="simulated_data_export/studies_fetus_train_train.csv",
    np_train="simulated_data_export/studies_fetus_train_train.npy",
    csv_valid="simulated_data_export/studies_fetus_train_test.csv",
    np_valid="simulated_data_export/studies_fetus_train_test.npy",
    csv_test="simulated_data_export/studies_fetus_test.csv",
    np_test="simulated_data_export/studies_fetus_test.npy",
    num_samples_train=5000,
    rescale_factor=10,
    batch_size=1
    )
dm.setup()

In [ ]:
dl_test = dm.test_dataloader()
it = iter(dl_test)

In [ ]:
X, X_origin, X_end, X_PC = next(it)
X = X[0:1]
X_origin = X_origin[0:1] 
X_end = X_end[0:1]
X_PC = X_PC[0:1]

In [ ]:

pc_head_k = knn_gather(X_PC, head_idx).squeeze(2)
pc_belly_k = knn_gather(X_PC, belly_idx).squeeze(2)
pc_side_k = knn_gather(X_PC, side_idx).squeeze(2)

points = torch.stack([torch.mean(pc_belly_k, dim=1),
                      torch.mean(pc_head_k, dim=1),
                      torch.mean(pc_side_k, dim=1)], dim=1)

frame = compute_orthogonal_frame(points)
plot_diffusion(X_PC.cpu().numpy(), points.cpu().numpy(), frame.cpu().numpy(), scale=10)

In [ ]:
# np.save("/mnt/raid/C1_ML_Analysis/simulated_data_export/fetus_rest_selected/head_idx.npy", head_idx.cpu().numpy())
# np.save("/mnt/raid/C1_ML_Analysis/simulated_data_export/fetus_rest_selected/belly_idx.npy", belly_idx.cpu().numpy())
# np.save("/mnt/raid/C1_ML_Analysis/simulated_data_export/fetus_rest_selected/side_idx.npy", side_idx.cpu().numpy())